# batches 

In [1]:
import os
import shutil

# Specify the directory where the images are stored
src_folder = r"C:\Users\Admin\Downloads\images-20240918T052943Z-001\images"

# Specify the destination folder where batches will be created
dst_folder =r"C:\Users\Admin\Downloads\dataset\batch_input"

# Create destination folders for the batches
batch_size = 2000
num_batches = 2

# Create destination directories for the batches
for i in range(1, num_batches + 1):
    os.makedirs(os.path.join(dst_folder, f"batch_{i}"), exist_ok=True)

# Get a list of all image files in the source directory
all_images = [f for f in os.listdir(src_folder) if os.path.isfile(os.path.join(src_folder, f))]

# Sort images to maintain a consistent order
all_images.sort()

# Loop over the images and move them into respective batch folders
for i, image in enumerate(all_images):
    batch_num = (i // batch_size) + 1
    if batch_num > num_batches:
        break  # Stop if we've reached the maximum number of batches
    # Move the image to the corresponding batch folder
    shutil.move(os.path.join(src_folder, image), os.path.join(dst_folder, f"batch_{batch_num}", image))

print("Images have been segregated into batches.")


Images have been segregated into batches.


# Normalize 0-1

In [2]:
import os
from PIL import Image
import numpy as np

# Directory containing the images to be normalized
input_folder = r"C:\Users\Admin\Downloads\dataset\batch_input\batch_1"
# Directory to save normalized images
output_folder = r"C:\Users\Admin\Downloads\dataset\batch_input\normz_input"

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Normalize function to convert image pixel values to range [0, 1]
def normalize_image(image):
    # Convert the image to a NumPy array
    image_array = np.asarray(image, dtype=np.float32)
    
    # Normalize the array values to the range [0, 1]
    normalized_array = image_array / 255.0
    
    # Convert back to Image object
    normalized_image = Image.fromarray((normalized_array * 255).astype(np.uint8))
    
    return normalized_image

# Loop through each image in the input folder
for image_name in os.listdir(input_folder):
    if image_name.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):  # Add more extensions if needed
        # Open the image
        image_path = os.path.join(input_folder, image_name)
        image = Image.open(image_path)
        
        # Normalize the image
        normalized_image = normalize_image(image)
        
        # Save the normalized image in the output folder
        normalized_image.save(os.path.join(output_folder, image_name))

print("Images normalized and saved successfully!")


Images normalized and saved successfully!


# check Anomaly by frequency 

In [6]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from skimage import io
from sklearn.cluster import KMeans
from matplotlib import cm

# Set up font size and plot settings
fontSize = 15
plt.rcParams.update({'font.size': fontSize})

#===============================================================================
# Folder paths
import os

# Folder paths
input_folder = r"C:\Users\Admin\Downloads\dataset\good"  # Folder containing images
output_folder = r"C:\Users\Admin\Downloads\dataset\old_good"  # Folder to save scatter plots

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Continue with the rest of the script...


# Function to process and generate scatter plot for a single image
def process_image(image_path, output_folder):
    # Read in the image
    originalRGBImage = io.imread(image_path)

    # Set up a subplot of 2 rows and 3 columns
    fig, axs = plt.subplots(2, 3, figsize=(15, 10))

    # Display the original image
    axs[0, 0].imshow(originalRGBImage)
    axs[0, 0].axis('on')
    caption = f'Original Pseudocolor Image, {os.path.basename(image_path)}'
    axs[0, 0].set_title(caption)

    # Convert the image to grayscale using the minimum value across the color channels
    grayImage = np.min(originalRGBImage, axis=2)

    # Crop off the surrounding clutter to get the colorbar
    colorBarImage = originalRGBImage[45:45+249, 533:533+13, :]

    # Crop off the surrounding clutter to get the RGB image
    rgbImage = originalRGBImage[40:40+259, 20:20+441, :]

    # Get the dimensions of the cropped RGB image
    rows, columns, numberOfColorChannels = rgbImage.shape

    # Display the cropped RGB image
    axs[0, 1].imshow(rgbImage)
    axs[0, 1].axis('on')
    caption = 'Cropped Pseudocolor Image'
    axs[0, 1].set_title(caption)

    # Display the colorbar image
    axs[0, 2].imshow(colorBarImage)
    axs[0, 2].axis('on')
    caption = 'Cropped Colorbar Image'
    axs[0, 2].set_title(caption)

    # Set up figure properties (adjust window and remove unnecessary toolbars)
    fig.subplots_adjust(top=0.9, bottom=0.05, left=0.05, right=0.95, hspace=0.3, wspace=0.3)
    fig.suptitle('Demo by ImageAnalyst', fontsize=20)

    # Get the color map from the color bar image
    storedColorMap = colorBarImage[:, 0, :].astype(float) / 255.0

    # Flatten the RGB image to apply KMeans clustering for quantization
    flattened_rgb = rgbImage.reshape((-1, 3))

    # Perform KMeans clustering to create an indexed image
    n_colors = len(storedColorMap)  # Number of colors based on the color bar
    kmeans = KMeans(n_clusters=n_colors, random_state=0).fit(flattened_rgb)
    labels = kmeans.predict(flattened_rgb)

    # Reshape the labels to form the indexed image
    indexedImage = labels.reshape(rgbImage.shape[:2])

    # Display the indexed image (gray scale)
    axs[1, 0].imshow(indexedImage, cmap=cm.gray)
    axs[1, 0].axis('on')
    caption = 'Indexed Image (Gray Scale Thermal Image)'
    axs[1, 0].set_title(caption)

    # Generate the histogram data
    hist_values, bins = np.histogram(indexedImage.ravel(), bins=n_colors, density=True)
    bin_centers = (bins[:-1] + bins[1:]) / 2

    # Scatter plot of the histogram data
    axs[1, 2].scatter(bin_centers, hist_values, color='blue', edgecolor='black')
    axs[1, 2].grid(True)
    caption = 'Scatter Plot of Thermal Image Data'
    axs[1, 2].set_title(caption)
    axs[1, 2].set_xlabel('Gray Scale')
    axs[1, 2].set_ylabel('Frequency')

    # Check the 0th bin (grayscale level 0) for defect detection
    defect_message = ""

    # If the 0th bin has a non-zero value, mark it as a defect
    if hist_values[0] > 0:
        defect_message = "Defect detected (0th grayscale value is non-zero)"
        print(defect_message)
        axs[1, 1].text(bin_centers[0], max(hist_values) * 0.9, defect_message, fontsize=19, color='red')
    else:
        defect_message = "No defect detected (0th grayscale value is zero)"
        print(defect_message)
        axs[1, 1].text(bin_centers[0], max(hist_values) * 0.9, defect_message, fontsize=12, color='green')

    # Save the scatter plot as a file
    output_image_path = os.path.join(output_folder, f"{os.path.basename(image_path).split('.')[0]}_scatter_plot.png")
    plt.savefig(output_image_path)

    # Close the plot to free memory
    plt.close()

#===============================================================================
# Loop through each image in the input folder and process it
for image_file in os.listdir(input_folder):
    if image_file.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):  # Add more extensions if needed
        image_path = os.path.join(input_folder, image_file)
        process_image(image_path, output_folder)

print("Processing complete! Scatter plots have been saved.")


C:\Users\Admin\anaconda3\Lib\site-packages\sklearn\base.py:1473: ConvergenceWarning: Number of distinct clusters (11) found smaller than n_clusters (249). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)


Defect detected (0th grayscale value is non-zero)


C:\Users\Admin\anaconda3\Lib\site-packages\sklearn\base.py:1473: ConvergenceWarning: Number of distinct clusters (12) found smaller than n_clusters (249). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)


Defect detected (0th grayscale value is non-zero)


C:\Users\Admin\anaconda3\Lib\site-packages\sklearn\base.py:1473: ConvergenceWarning: Number of distinct clusters (11) found smaller than n_clusters (249). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)


...

Defect detected (0th grayscale value is non-zero)


C:\Users\Admin\anaconda3\Lib\site-packages\sklearn\base.py:1473: ConvergenceWarning: Number of distinct clusters (12) found smaller than n_clusters (249). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)


Defect detected (0th grayscale value is non-zero)


C:\Users\Admin\anaconda3\Lib\site-packages\sklearn\base.py:1473: ConvergenceWarning: Number of distinct clusters (12) found smaller than n_clusters (249). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)


Defect detected (0th grayscale value is non-zero)
Processing complete! Scatter plots have been saved.
